In [155]:
DATASET_NAME = "UC3M-LCPM/DIPROMATS_2024"
DATASET_SPLIT = "task1a_aug"

In [156]:
ES_MODEL_NAME = "UC3M-LCPM/Beto_base_uncased_task1a_es_aug"
EN_MODEL_NAME = "UC3M-LCPM/Bert_base_uncased_task1a_en_aug"

In [115]:
SELECTED_COLUMNS = ['label', 'text']

In [116]:
labels_ids = {'False': 0, 'True': 1}
n_labels = len(labels_ids)

# Login to drive

In [5]:
from google.colab import drive
# mount your google drive
drive.mount('/content/drive')

# cambiamos el directorio de trabajo a la carpeta donde está almacenado el dataset
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/Modelos/')

Mounted at /content/drive


# Install dependencies

In [6]:
!pip install -q transformers[torch] datasets
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 43.8 MB/s eta 0:00:00


# Login to HuggingFace

In [7]:
from huggingface_hub import HfFolder, notebook_login

notebook_login()

# Load dataset

In [157]:
from datasets import load_dataset, Dataset

dataset = load_dataset(DATASET_NAME, DATASET_SPLIT)

Generating train split:   0%|          | 0/34677 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/2179 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2180 [00:00<?, ? examples/s]

In [158]:
dataset = dataset.rename_column('label_task1', 'label')

In [159]:
dataset["train"]

Dataset({
    features: ['test_case', 'id', 'country', 'username', 'tweet_type', 'tweet_id', 'UTC', 'rts&fav', 'language', 'text', 'label', 'label_task2', '0 not propagandistic', '1 appeal to commonality', '2 discrediting the opponent', '3 loaded language', 'label_task3', '__index_level_0__'],
    num_rows: 34677
})

# Define metrics function

In [160]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def compute_metrics(pred):

    y_true = pred.label_ids                 # son las labels reales
    y_pred = pred.predictions.argmax(-1)    # son las predicciones


    acc = accuracy_score(y_true, y_pred)

    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# SPANISH MODEL

### Clean dataset columns

In [161]:
COLS_TO_REMOVE = [column for column in dataset['train'].column_names if column not in SELECTED_COLUMNS]

In [162]:
dataset_train_es = dataset['train'].filter(lambda example: example['language'] == 'es')
dataset_train_es = dataset_train_es.remove_columns(COLS_TO_REMOVE)
dataset_train_es

Filter:   0%|          | 0/34677 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 14677
})

In [163]:
dataset_val_es = dataset['val'].filter(lambda example: example['language'] == 'es')
dataset_val_es = dataset_val_es.remove_columns(COLS_TO_REMOVE)
dataset_val_es

Filter:   0%|          | 0/2179 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 893
})

### Load model

In [164]:
from transformers import (AutoConfig,
                          AutoTokenizer,
                          AutoModelForSequenceClassification,
                          TrainingArguments,
                          pipeline,
                          set_seed)

set_seed(123)

epochs = 4
batch_size = 32

MAX_LENGTH = 60

model_name_or_path = "dccuchile/bert-base-spanish-wwm-uncased"

In [165]:
model_config = AutoConfig.from_pretrained(pretrained_model_name_or_path=model_name_or_path, num_labels=n_labels)

tokenizer_es = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name_or_path)
tokenizer_es.pad_token = tokenizer_es.eos_token
if tokenizer_es.pad_token is None:
    tokenizer_es.add_special_tokens({'pad_token': '[PAD]'})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [166]:
model_es = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name_or_path, config=model_config).to("cuda")

# resize model embedding to match new tokenizer
model_es.resize_token_embeddings(len(tokenizer_es))

# fix model padding token id
model_es.config.pad_token_id = model_es.config.eos_token_id

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Tokenize datasets

In [167]:
tokenized_ids = [labels_ids[str(label)] for label in dataset_train_es["label"]]
print(tokenized_ids)
dataset_train_es = dataset_train_es.remove_columns("label").add_column("label", tokenized_ids)

[1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 

Flattening the indices:   0%|          | 0/14677 [00:00<?, ? examples/s]

In [168]:
tokenized_ids = [labels_ids[str(label)] for label in dataset_val_es["label"]]
print(tokenized_ids)
dataset_val_es = dataset_val_es.remove_columns("label").add_column("label", tokenized_ids)

[0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 

Flattening the indices:   0%|          | 0/893 [00:00<?, ? examples/s]

In [169]:
def tokenize(examples):
    return tokenizer_es(examples["text"], return_tensors="pt", padding="max_length", truncation=True, max_length=MAX_LENGTH).to("cuda")

In [170]:
encoded_data_train = dataset_train_es.map(tokenize, batched=True)
encoded_data_train

Map:   0%|          | 0/14677 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 14677
})

In [171]:
encoded_data_val = dataset_val_es.map(tokenize, batched=True)
encoded_data_val

Map:   0%|          | 0/893 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 893
})

### Configure model

In [172]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="modelos/" + ES_MODEL_NAME,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    logging_dir=f"modelos/{ES_MODEL_NAME}/logs",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard",
    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=ES_MODEL_NAME,
    hub_token=HfFolder.get_token(),
)

### Train model

In [173]:
from transformers import Trainer

trainer = Trainer(
    model = model_es,            # modelo que será ajustado
    train_dataset = encoded_data_train, # conjunto training
    eval_dataset = encoded_data_val,   # conjunto de validación

    args = training_args,     # hiperparámetros
    compute_metrics=compute_metrics,    # función para computar las métricas
)

In [174]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.205400,0.426224,0.839866,0.777051,0.766133,0.791065
2,0.032700,0.791114,0.855543,0.785620,0.790705,0.780938
3,0.004000,0.889081,0.858903,0.791002,0.795567,0.786757
4,0.008600,0.924391,0.862262,0.780985,0.813200,0.759571


TrainOutput(global_step=1836, training_loss=0.1362705227212822, metrics={'train_runtime': 198.8289, 'train_samples_per_second': 295.269, 'train_steps_per_second': 9.234, 'total_flos': 1810162949774400.0, 'train_loss': 0.1362705227212822, 'epoch': 4.0})

In [175]:
trainer.evaluate()

{'eval_loss': 0.42622360587120056,
 'eval_accuracy': 0.8398656215005599,
 'eval_f1': 0.7770512925714026,
 'eval_precision': 0.7661334582691921,
 'eval_recall': 0.7910652065704331,
 'eval_runtime': 0.8883,
 'eval_samples_per_second': 1005.262,
 'eval_steps_per_second': 31.52,
 'epoch': 4.0}

### Push model to HuggingFace

In [176]:
model_es.push_to_hub(ES_MODEL_NAME, private=True)

CommitInfo(commit_url='https://huggingface.co/UC3M-LCPM/Beto_base_uncased_task1a_es_aug/commit/3b053e8a1ee1607e9719450735bd975396fccd9a', commit_message='Upload BertForSequenceClassification', commit_description='', oid='3b053e8a1ee1607e9719450735bd975396fccd9a', pr_url=None, pr_revision=None, pr_num=None)

In [177]:
tokenizer_es.push_to_hub(ES_MODEL_NAME, private=True)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/UC3M-LCPM/Beto_base_uncased_task1a_es_aug/commit/2e2ec29438125138aa73dc6c76d8fc25481a0050', commit_message='Upload tokenizer', commit_description='', oid='2e2ec29438125138aa73dc6c76d8fc25481a0050', pr_url=None, pr_revision=None, pr_num=None)

# ENGLISH MODEL

### Clean dataset columns

In [178]:
COLS_TO_REMOVE = [column for column in dataset['train'].column_names if column not in SELECTED_COLUMNS]

In [179]:
dataset_train_en = dataset['train'].filter(lambda example: example['language'] == 'en')
dataset_train_en = dataset_train_en.remove_columns(COLS_TO_REMOVE)
dataset_train_en

Filter:   0%|          | 0/34677 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 20000
})

In [180]:
dataset_val_en = dataset['val'].filter(lambda example: example['language'] == 'en')
dataset_val_en = dataset_val_en.remove_columns(COLS_TO_REMOVE)
dataset_val_en

Filter:   0%|          | 0/2179 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 1286
})

### Load model

In [181]:
from transformers import (BertConfig,
                          BertTokenizer,
                          BertForSequenceClassification,
                          TrainingArguments,
                          pipeline,
                          set_seed)

set_seed(123)

epochs = 4
batch_size = 32

MAX_LENGTH = 50

model_name_or_path = "google-bert/bert-base-uncased"

In [182]:
model_config = BertConfig.from_pretrained(pretrained_model_name_or_path=model_name_or_path, num_labels=n_labels)

tokenizer_en = BertTokenizer.from_pretrained(pretrained_model_name_or_path=model_name_or_path)
tokenizer_en.pad_token = tokenizer_en.eos_token
if tokenizer_en.pad_token is None:
    tokenizer_en.add_special_tokens({'pad_token': '[PAD]'})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [183]:
model_en = BertForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name_or_path, config=model_config).to("cuda")

# resize model embedding to match new tokenizer
model_en.resize_token_embeddings(len(tokenizer_en))

# fix model padding token id
model_en.config.pad_token_id = model_en.config.eos_token_id

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Tokenize datasets

In [184]:
tokenized_ids = [labels_ids[str(label)] for label in dataset_train_en["label"]]
print(tokenized_ids)
dataset_train_en = dataset_train_en.remove_columns("label").add_column("label", tokenized_ids)

[0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 

Flattening the indices:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [185]:
tokenized_ids = [labels_ids[str(label)] for label in dataset_val_en["label"]]
print(tokenized_ids)
dataset_val_en = dataset_val_en.remove_columns("label").add_column("label", tokenized_ids)

[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 

Flattening the indices:   0%|          | 0/1286 [00:00<?, ? examples/s]

In [186]:
def tokenize(examples):
    return tokenizer_en(examples["text"], return_tensors="pt", padding="max_length", truncation=True, max_length=MAX_LENGTH).to("cuda")

In [187]:
encoded_data_train = dataset_train_en.map(tokenize, batched=True)
encoded_data_train

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 20000
})

In [188]:
encoded_data_val = dataset_val_en.map(tokenize, batched=True)
encoded_data_val

Map:   0%|          | 0/1286 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1286
})

### Configure model

In [189]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="modelos/" + EN_MODEL_NAME,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    logging_dir=f"modelos/{EN_MODEL_NAME}/logs",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard",
    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=EN_MODEL_NAME,
    hub_token=HfFolder.get_token(),
)

### Train model

In [190]:
from transformers import Trainer

trainer = Trainer(
    model = model_en,            # modelo que será ajustado
    train_dataset = encoded_data_train, # conjunto training
    eval_dataset = encoded_data_val,   # conjunto de validación

    args = training_args,     # hiperparámetros
    compute_metrics=compute_metrics,    # función para computar las métricas
)

In [191]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.173800,0.516733,0.813375,0.755420,0.737469,0.791438
2,0.086600,0.796622,0.838258,0.769045,0.761785,0.777529
3,0.082900,0.844121,0.848367,0.757576,0.785886,0.738638
4,0.003000,0.942174,0.844479,0.760332,0.774416,0.749111


TrainOutput(global_step=2500, training_loss=0.14163471769299357, metrics={'train_runtime': 228.5578, 'train_samples_per_second': 350.021, 'train_steps_per_second': 10.938, 'total_flos': 2055555120000000.0, 'train_loss': 0.14163471769299357, 'epoch': 4.0})

In [192]:
trainer.evaluate()

{'eval_loss': 0.5167328715324402,
 'eval_accuracy': 0.8133748055987559,
 'eval_f1': 0.7554203119056676,
 'eval_precision': 0.7374690042197765,
 'eval_recall': 0.7914384256441469,
 'eval_runtime': 1.0757,
 'eval_samples_per_second': 1195.5,
 'eval_steps_per_second': 38.115,
 'epoch': 4.0}

### Push model to HuggingFace

In [193]:
model_en.push_to_hub(EN_MODEL_NAME, private=True)

CommitInfo(commit_url='https://huggingface.co/UC3M-LCPM/Bert_base_uncased_task1a_en_aug/commit/8d5f1ffc3be9f338537f5974d47b9bed35662719', commit_message='Upload BertForSequenceClassification', commit_description='', oid='8d5f1ffc3be9f338537f5974d47b9bed35662719', pr_url=None, pr_revision=None, pr_num=None)

In [194]:
tokenizer_en.push_to_hub(EN_MODEL_NAME, private=True)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/UC3M-LCPM/Bert_base_uncased_task1a_en_aug/commit/0e2cbb51ac1f2296532032be085d4a770a89eebc', commit_message='Upload tokenizer', commit_description='', oid='0e2cbb51ac1f2296532032be085d4a770a89eebc', pr_url=None, pr_revision=None, pr_num=None)